In [1]:
import os
import re
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import torch
import torchvision.transforms.functional as TF
import tifffile
import yaml
from einops import rearrange


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.data.multiplex import extract_ome_tiff, write_basic_ome_tiff, get_ome_tiff_channels, make_pseudo
from mushroom.data.visium import adata_from_visium
from mushroom.visualization.utils import display_sections
import mushroom.utils as utils

In [5]:
run_dir = '/data/estorrs/mushroom/data/projects/htan_talk'
mushroom_dir = os.path.join(run_dir, 'mushroom')
Path(mushroom_dir).mkdir(parents=True, exist_ok=True)

In [6]:
metadata = yaml.safe_load(open(os.path.join(run_dir, 'registered', 'metadata_v2.yaml')))
metadata

{'HT413C1_Th1k4A1': {'data': {'cosmx': {'HT413C1_Th1k4A1_U34': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s21_cosmx.h5ad',
    'HT413C1_Th1k4A1_U7': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s8_cosmx.h5ad'},
   'he': {'HT413C1_Th1k4A1_U1': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s4_he.tif',
    'HT413C1_Th1k4A1_U11': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s11_he.tif',
    'HT413C1_Th1k4A1_U14': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s0_he.tif',
    'HT413C1_Th1k4A1_U18': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s1_he.tif',
    'HT413C1_Th1k4A1_U21': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s12_he.tif',
    'HT413C1_Th1k4A1_U24

In [7]:
configs = {}
for case, data in metadata.items():
    config = [
        {
            'id': sid,
            'position': 0,
            'data': []
        }
        for sid in data['ids']
    ]
    
    for dtype, mapping in data['data'].items():
        for sample_id, filepath in mapping.items():
            idx = data['order'].index(sample_id)
            
            # convert from katmai to glacier
            filepath = filepath.replace('/diskmnt/Projects/Users/estorrs', '/data/estorrs')
            
            config[idx]['data'].append(
                {
                    'dtype': dtype,
                    'filepath': filepath,
                }
            )
    configs[case] = config
configs
    

{'HT413C1_Th1k4A1': [{'id': 's0',
   'position': 0,
   'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s0_he.tif'}]},
  {'id': 's1',
   'position': 0,
   'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s1_he.tif'}]},
  {'id': 's2',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s2_xenium.h5ad'}]},
  {'id': 's3',
   'position': 0,
   'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s3_multiplex.ome.tiff'}]},
  {'id': 's4',
   'position': 0,
   'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s4_he.tif'}]},
  {'id': 's5',
   'position': 0,
   'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/pr

In [8]:
import sys
yaml.safe_dump(configs, sys.stdout)

HT225C1:
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT225C1/s0_visium.h5ad
  id: s0
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT225C1/s1_visium.h5ad
  id: s1
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT225C1/s2_visium.h5ad
  id: s2
  position: 0
- data:
  - dtype: visium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT225C1/s3_visium.h5ad
  id: s3
  position: 0
HT413C1_Th1k4A1:
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s0_he.tif
  id: s0
  position: 0
- data:
  - dtype: he
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A1/s1_he.tif
  id: s1
  position: 0
- data:
  - dtype: xenium
    filepath: /data/estorrs/mushroom/data/projects/htan_talk/registered/HT413C1_Th1k4A

In [9]:
os.path.join(mushroom_dir, 'configs_template.yaml')

'/data/estorrs/mushroom/data/projects/htan_talk/mushroom/configs_template.yaml'

In [19]:
# # filling out position seperately and reading back in
# yaml.safe_dump(configs, open(os.path.join(mushroom_dir, 'configs_template.yaml'), 'w'))

In [10]:
configs = yaml.safe_load(open(os.path.join(mushroom_dir, 'configs_template.yaml')))
configs

{'S18-9906': [{'data': [{'dtype': 'he',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_he.tif'},
    {'dtype': 'visium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s0_visium.h5ad'}],
   'id': 's0',
   'position': 0},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s1_xenium.h5ad'}],
   'id': 's1',
   'position': 5},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s2_multiplex.ome.tiff'}],
   'id': 's2',
   'position': 10},
  {'data': [{'dtype': 'xenium',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s3_xenium.h5ad'}],
   'id': 's3',
   'position': 40},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/htan_talk/registered/S18-9906/s4_multiplex.ome.tiff'}],
   'id': 's4',
   'position': 45

In [11]:
for case, config in configs.items():
    directory = os.path.join(mushroom_dir, case)
    Path(directory).mkdir(parents=True, exist_ok=True)
    
    yaml.safe_dump(config, open(os.path.join(directory, 'config.yaml'), 'w'))

In [12]:
mushroom_dir

'/data/estorrs/mushroom/data/projects/htan_talk/mushroom'